In [1]:
import os, sys
import argparse
from biobb_io.api.pdb import pdb
from biobb_amber.pdb4amber.pdb4amber_run import pdb4amber_run
from biobb_amber.leap.leap_gen_top import leap_gen_top
from biobb_amber.pmemd.pmemd_mdrun import pmemd_mdrun
from biobb_amber.process.process_minout import process_minout
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb
from biobb_amber.leap.leap_solvate import leap_solvate
from biobb_amber.leap.leap_add_ions import leap_add_ions
from biobb_amber.process.process_mdout import process_mdout
from biobb_amber.pmemd.pmemd_mdrun import pmemd_mdrun

import json

In [2]:
def create_dictionaries(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    dictionaries = {}
    for key in data.keys():
        dictionaries[key] = data[key]
    return dictionaries

In [ ]:
def prepPDB(input_PDB, forcefield, destination="./"):
        
    ## Clean the PDB file using pdb4amber tool
    output_pdb4amber_path = os.path.join(destination,'structure.pdb4amber.pdb')
    pdb4amber_run(input_pdb_path=input_PDB, output_pdb_path=output_pdb4amber_path, properties=forcefield)
    
    ## Create topology files for the protein. 
    leap_gen_top (input_pdb_path=output_pdb4amber_path,
                  
                  output_pdb_path=os.path.join(destination,'structure.leap.pdb'), 
                  output_top_path=os.path.join(destination,'structure.leap.top'), 
                  output_crd_path=os.path.join(destination,'structure.leap.crd'), 
                  properties=forcefield)

In [ ]:
force field = ["protein.ff19SB", "lipid21", "water.tip3p"]

def prepSystem(input_PDB, forcefield, destination="./"):
    
    ## Create topology files for the protein. 
    leap_gen_top (input_pdb_path=input_PDB,
                  input_params_path="frcmod.ionsjc_tip3p",
                  input_frcmod_path="./ligand_params/missing_gaff2.frcmod",
                  input_lib_path="./ligand_params/IXO.off",
                  output_pdb_path=os.path.join(destination,'structure.leap.pdb'), 
                  output_top_path=os.path.join(destination,'structure.leap.top'), 
                  output_crd_path=os.path.join(destination,'structure.leap.crd'), 
                  properties=forcefield)

In [ ]:
def mdrun (stage, input_top, prop, destination="../example/"):

    ## PMEMD_mdrun for any steps, including Minimization, Heating, NVT, NPT, and Production
    output_traj_path = os.path.join(destination,f"pmemd.{stage}.nc")
    output_rst_path = os.path.join(destination,f"pmemd.{stage}.rst")
    output_log_path = os.path.join(destination,f"pmemd.{stage}.log")
    
    print(output_traj_path, output_rst_path, output_log_path)

    ## Create and launch bb
    pmemd_mdrun(input_top_path=os.path.join(destination,input_top["top"]),
                 input_crd_path=os.path.join(destination,input_top["crd"]),
                 input_ref_path=os.path.join(destination,input_top["crd"]),
                 output_traj_path=output_traj_path,
                 output_rst_path=output_rst_path,
                 output_log_path=output_log_path,
                 properties=prop)

In [ ]:
inputs_dict = create_dictionaries('../example/6vn7/inputs.json')

In [ ]:
input_ions_top = {
    "top": "structure.ions.parmtop",
    "crd": "structure.ions.crd",
    "pdb": "structure.ions.pdb"
}

prop_min = {
    "simulation_type" : "minimization",
    "mdin" : { 
        'maxcyc' : 300, # Reducing the number of minimization steps for the sake of time
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 50.0                       # With a force constant of 50 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("min", input_ions_top, prop_min, destination="../example/1aki/")

In [ ]:
input_heat_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.min.log",
    "rst": "pmemd.min.rst",
    "crd": "pmemd.min.rst"
}

prop_heat = {
    "simulation_type" : "heat",
    "binary_path": "/nfs_home/software/amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 2500, # Reducing the number of steps for the sake of time (5ps)
        'ntr' : 1,       # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 10.0                       # With a force constant of 10 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("heat", input_heat_top, prop_heat, destination="../example/1aki/")

In [ ]:
input_nvt_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.heat.log",
    "rst": "pmemd.heat.rst",
    "crd": "pmemd.heat.rst"
}


prop_nvt = {
    "simulation_type" : 'nvt',
    "remove_tmp": False,
    "binary_path": "/nfs_home/software/amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 50000, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 5.0,                            # With a force constant of 5 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("nvt", input_nvt_top, prop_nvt, destination="../example/1aki/")

In [ ]:
input_npt_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.nvt.log",
    "rst": "pmemd.nvt.rst",
    "crd": "pmemd.nvt.rst"
}

prop_npt = {
    "simulation_type" : 'npt',
    "binary_path": "/nfs_home/software/amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 50000, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 2.5                               # With a force constant of 2.5 Kcal/mol*A2
    }
}

In [ ]:
mdrun ("npt", input_npt_top, prop_npt, destination="../example/1aki/")

In [ ]:
input_free_top = {
    "top": "structure.ions.parmtop",
    "log": "pmemd.npt.log",
    "rst": "pmemd.npt.rst",
    "crd": "pmemd.npt.rst"
}

prop_free = {
    "simulation_type" : 'free',
    "binary_path": "/nfs_home/software/amber/22/bin/pmemd.cuda",
    "mdin" : { 
        'nstlim' : 250000, # Reducing the number of steps for the sake of time (5ps)
        'ntwx' : 500  # Print coords to trajectory every 500 steps (1 ps)
    }
}

In [ ]:
mdrun ("free", input_free_top, prop_free, destination="../example/1aki/")